In [ ]:
# Set this to wherever you have your LV rxn data stored
home = %env HOME
lv_rxn_data = "{}/lv/lv_files/mcell/output_data/react_data/".format(home)
lv_rxn_data

In [ ]:
import glob
import pandas as pd
import ipywidgets as widgets
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go

# Set offline to true or you'll have to create a plotly account
cf.set_config_file(offline=True, world_readable=False, theme='pearl')
py.init_notebook_mode() # graphs charts inline

In [ ]:
# Create a list of dataframes, one for every "prey" reaction data output file
# e.g., ./seed_0001/prey.World.dat, ./seed_0002/prey.World.dat, etc

prey_fnames = list(glob.iglob("{}/**/prey.World.dat".format(lv_rxn_data)))
prey_fnames.sort()

df_list = []
for fname in prey_fnames:
    seed = int(fname.split('/')[-2][5:])
    df = pd.read_csv(fname, names=['time', "s{}".format(seed)], delim_whitespace=True)
    df = df.set_index('time')
    df_list.append(df)

In [ ]:
# Turn all the dataframes into a single dataframe
df_all_prey = pd.concat(df_list, axis=1)
df_all_prey.head()

In [ ]:
# To make loc indexing work better, we'll remove the time column
# We can save the time column to a series in case we want it later
df_all_prey = df_all_prey.reset_index()
series_time = df_all_prey.pop('time')

In [ ]:
# Just an example of loc slicing
df_all_prey.loc[450:500:10][['s1', 's2']]

In [ ]:
# The same example with iloc slicing
df_all_prey.iloc[450:501:10, :2]

In [ ]:
df_all_prey.iplot(
    xTitle="Time (s)",
    yTitle="Counts",
    title="MCell Simulation of Lotka-Volterra System")

In [ ]:
# Standard deviation for all prey
std_all_prey = df_all_prey.std(axis=1)

In [ ]:
# Simple plot of mean
prey_avg = df_all_prey.mean(axis=1)
prey_avg.iplot(
    xTitle="Time (μs)",
    yTitle="Counts",
    title="Average of Lotka-Volterra MCell Sim.")

In [ ]:
# There are many ways to customize your plot

data = [go.Scatter(
    x=df_all_prey.index,
    y=prey_avg,
    name='Prey Average',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,
        dash = 'dash'),
    hoverinfo='name',
)]

layout=go.Layout(
    title="Average of Lotka-Volterra MCell Sim.",
    paper_bgcolor='rgb(255,255,255)',
    plot_bgcolor='rgb(229,229,229)',
    xaxis=go.XAxis(
        title='Time (μs)',
        gridcolor='rgb(255,255,255)',
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False
    ),
    yaxis=go.YAxis(
        title='Molecule Counts',
        gridcolor='rgb(255,255,255)',
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False
    ),
)

py.iplot(go.Figure(data=data, layout=layout))